# Import Required Libraries

In [ ]:
import re
import pandas as pd
import numpy as np

# Import Dataset

In [ ]:
dataset=pd.read_csv("Reviews.csv")

# To Check FIrst 5 observations

In [ ]:
dataset.head()

# To Check Last 5 observations

In [ ]:
dataset.tail()

# Checking For Null Values

In [ ]:
dataset.isnull().any()

# Fill null values

In [ ]:
dataset['body'] = dataset['body'].fillna('').apply(str)
dataset['name'] = dataset['name'].fillna('').apply(str)
dataset['title'] = dataset['title'].fillna('').apply(str)
dataset['helpfulVotes'] = dataset['helpfulVotes'].fillna('').apply(str)

# Drop Null values

In [ ]:
dataset.isnull().sum()

# Dropping columns

In [ ]:
dataset=dataset.drop(columns=['asin','name','helpfulVotes','date'],axis=1)

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.head()

In [ ]:
a=dataset['rating'].tolist()
a

### Based on the rating appending value 0 or 1 to the empty list

In [ ]:
d=[]
for i in range(len(a)):
    if a[i]>=3:
        d.append(1)
    else:
        d.append(0)
d

In [ ]:
dt=pd.DataFrame(d,columns=['emotion'])
dt

In [ ]:
data1=pd.concat([dataset,dt],axis=1)
data1.head()

### Drop verified column from data1

In [ ]:
data1.drop(['verified'],axis=1,inplace=True)

### Join both title and body

In [ ]:
data1['Review'] = data1[['title', 'body',]].agg(' '.join, axis=1)

In [ ]:
data1.head()

Dropping Columns

In [ ]:
data1.drop(['title','body','rating'],axis=1,inplace=True)

In [ ]:
data1.head()

In [ ]:
len(list(data1['emotion']))

split the data into x(independent variable)

In [ ]:
x=data1.iloc[:,1].values

In [ ]:
len(x)

In [ ]:
data1.head()

Check the data1 shape

In [ ]:
data1.shape

In [ ]:
#import natural language toolkit
import nltk
#import stopwords library to remove stopwords
from nltk.corpus import stopwords
#library used for stem the words
from nltk.stem.porter import PorterStemmer
#create an object for stemming
ps = PorterStemmer()
#library used for stem the words
from nltk.stem import WordNetLemmatizerWordNetLemmatizer
#create an object for wordnet lemmatizer
wordnet=WordNetLemmatizer()

In [ ]:
# Initialize empty array to append clean text  
corpus=[]
# no of rows to clean 
for i in range(len(x)):
    #replacing punctuations and numbers using re library 
    temp=re.sub('[^a-zA-Z]',' ',x[i])
    
    # convert all text to lower cases
    temp=temp.lower()
    # split to array(default delimiter is " ") 
    temp=temp.split()
    
    # creating WordNetLemmatizer object to take main lemma of each word
    wordnet = WordNetLemmatizer()
    #loop for leammatization each word in string array at ith row 
    temp=[wordnet.lemmatize(word) for word in temp if not word in set(
        stopwords.words('english'))]
    
    #rejoin all string array elements to create back into a string 
    temp=' '.join(temp)
    #append each string to create array of clean text 
    corpus.append(temp)

Before Stopwords: 
    "This is a sample sentence, showing off the stop words filtration."
    
After removing stopwords: 
    "This sample sentence, showing stop words filtration."

In [ ]:
corpus

In [ ]:
#creating bag of word model
from sklearn.feature_extraction.text import CountVectorizer
#To extract max 2000 feature, "max_features" is attribute to 
#experiment with to get better results
cv=CountVectorizer(max_features= 2000)
#z contains vectorized data (independent variable) 
z=cv.fit_transform(corpus).toarray()

Save the Bag of word model

In [ ]:
import pickle
pickle.dump(cv,open('count_vec.pkl','wb'))

In [ ]:
type(z)

In [ ]:
z

Creating dependent variable

In [ ]:
# y contains answers if review is positive or negative 
y=data1.iloc[:,0].values
y

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(z,y,test_size=0.33)

In [ ]:
x_train.shape

In [ ]:
#import library which uses tensorflow as backend
import keras
#sequential is to intialise the model
from keras.models import Sequential
#Dense is used to add layers
from keras.layers import Dense

In [ ]:
model=Sequential()

Adding Output Layer

In [ ]:
#adding output layer
model.add(Dense(init="random_uniform",activation="sigmoid",output_dim=1))

In [ ]:
model.add(Dense(input_dim=100,init="random_uniform",activation="relu",output_dim=30))
model.add(Dense(init="random_uniform",activation="relu",output_dim=25))
model.add(Dense(init="random_uniform",activation="sigmoid",output_dim=1))

compile the model

In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

Training the Model

In [ ]:
model.fit(x_train,y_train,batch_size=32,epochs=50)

Save the Model

In [ ]:
#this will save your model weights
#and h5 is the extension for keras
model.save('phone.h5')

In [ ]:
#importing load_model for loading h5 file
from keras.models import load_model
#importing numpy
import numpy as np
#importing pickle for load bag of word model
import pickle

#loading phone.h5 file using load_model
model=load_model('phone.h5')
#creating an empty array
corpus=[]
#open bag of word file
with open('count_vec.pkl','rb') as file:
    cv=pickle.load(file)
    
#Giving review
    inp = "This is a great, reliable phone"
    x=cv.transform([inp])
    y=model.predict(x)
    if(y>0.5):
        print('Positive review')
    else:
        print('Negative review')